# GAP Habitat Models as Species-Level Dictionaries
author: NMT

A script for developing a workflow to get model dictionaries onto ScienceBase.  This is now to a point where it works.  I have another more succinct copy of the script that I can run locally when it's time to upload all.   

Note: I created a sciencebase module in gapproduction with functions that connect to sciencebase (credentials saved in config file), return a list of species codes for species with sciencebase identifiers, return the ID, and return the doi for a passed species.

In [1]:
import gapproduction as gp
import pysb
import json
from IPython.display import display

In [2]:
# Where to save json files
outDir = "P:/Proj3/USGap/Vert/Model/2001/GAP_habitat_models/"

In [3]:
# Set up your ScienceBase session for operating against. 
sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

Username: "nmtarr@ncsu.edu"
········
Invalid password, try again
········


<pysb.SbSession.SbSession instance at 0x00000000059EE108>

Connect to ScienceBase and get a list of gap species codes from the list of child IDs for the habitat ma ID.

In [4]:
# This is the top level ScienceBase Item ID for the habitat maps
_gapHabitatMapCollectionItem = "527d0a83e4b0850ea0518326"

# This gets all the child IDs from the habitat map collection; returns a list
# you can loop through
habitatMapIDs = sb.get_child_ids(_gapHabitatMapCollectionItem)

# Get list of strUCs from habitatMapIDs to loop on
spUCs = [sb.get_item(x)["identifiers"][0]["key"] for x in habitatMapIDs[1:2]]
print(spUCs)

[u'mMOVOn']


The above cell can now be done with gapproduction modules.  I think they'll end up being common tasks. - NMT

# Draft dictionary - What needs to change?
The next cell creates the dictionaries and saves them as json files.
We'll need to make sure the format is good for everyone and fill in some blanks.  The keys are currently 'models', 'taxonomic', 'doi', 'input_layers', 'habitat_description', and 'references'

The next cell will likely be put in a function within gapproduction so it can be run with one line of code in top level scripts or modeling workflow scripts.

In [5]:
# Generate species model dictionaries and save as json files.
counter = 0
for species in spUCs:
    counter += 1
    print(species)
    print(str(counter) + " of " + str(len(spUCs)))
    
    # Make an empty dictionary to collect model dictionaries
    speciesDict = {"input_layers":gp.gapmodeling.layers_2001}
    
    # List taxanomic information
    taxonomic = {"common_name":gp.gapdb.NameCommon(species), 
                 "scientific_name":gp.gapdb.NameSci(species),
                 "gap_code": gp.gapdb.Crosswalk(species)[0],
                 "ELCode": gp.gapdb.Crosswalk(species)[1],
                 "ITIS_TSN": gp.gapdb.Crosswalk(species)[2],
                 "Global_SEQ_ID": gp.gapdb.Crosswalk(species)[3]}
    speciesDict["taxonomic"] = taxonomic
    
    # Add placeholder for DOI
    speciesDict["doi"] = gp.sciencebase.GetHabMapDOI(species)
    
    # Add the species' habitat description
    description = gp.gapmodeling.getHabitatDescription(species)
    speciesDict["habitat_description"] = description
    
    # Add species' references/citations
    referencesDF = gp.gapmodeling.SpReferences(species)
    references = dict(referencesDF["memCitation"])
    speciesDict["references"] = references
    
    # Establish a file name for speciesDict
    filename = outDir + species + "_CONUS_HabModel_2001v1.json"
    
    # Get python tuple (like a list) of regional model codes
    modelCodes = [x for x in gp.gapmodeling.ModelCodes(species, 
                                                 publishedOnly=True, 
                                                 conusOnly=True, 
                                                 migratory=False) if int(x[-1:]) in [1,2,3,4,5,6]]
    
    # Get the models as dictionaries, add to a dictionary of models, then add that
    # to the species level dictionary
    models = {}
    for model in modelCodes:
        modelDict = gp.gapmodeling.ModelAsDictionary(model, ecolSystem = "both")
        modelDict["intElevMax"] = 100 # !!!!!!!!!!!!!!!!!  REMOVE
        modelDict["intElevMin"] = 105 # !!!!!!!!!!!!!!!!!  REMOVE
        models[model] = modelDict
    
    speciesDict["models"] = models
    
    # Save species model dictionary as json object
    outfile = open(filename, "w")
    json.dump(speciesDict, outfile)
display(speciesDict)

mMOVOn
1 of 1


{'doi': u'doi:10.5066/F7SJ1J08',
 'habitat_description': u"Southwest:\r\nIn Nevada, M. montanus is taken in tule swamps, bogs, around springs, and never more than a few feet from water (New Mexico Department of Game and Fish 2000).  The species inhabits meadows in montane forests, open pine-oak forests, wet sedge and grass meadows bordering marshes and open water in New Mexico (Thompson et al. 1996). In Arizona, it is found in wet to damp places such as wet, grassy areas, bogs, and marshes (Hoffmeister 1986). It occurs in Colorado in drier grasslands with forbs and sagebrush, though fewer in number (Fitzgerald et al. 1994).  It is found in grama-galleta steppe - juniper-pinyon woodland mosaic in New Mexico (Thompson et al. 1996). PNV (BLM)-wet grassy meadows of yellow pine, red fir, Engelmann's spruce, hemlock, and lodgepole forests (Thompson et al. 1996).  In Nevada, the species prefers to live in ladino clover or strawberry clover rather than meadow fescue, when given a choice (Hall 

# Upload json dictionaries to ScienceBase habitat map items


In [6]:
# Tried the following and Knox Jones's Pocket Gopher.  They were successful.
species = "rRNSNx" # Ring-necked Snake
testID = '58fe2369e4b0f87f0854ae89'
item = sb.get_item(testID)
filename = outDir + species + "_CONUS_HabModel_2001v1.json"
sb.upload_file_to_item(item, filename, scrape_file=True)

{u'body': u'This dataset represents a species habitat distribution model for Ring-necked Snake.  These habitat maps are created by applying a <a href="https://www.sciencebase.gov/catalog/item/527d0a83e4b0850ea0518326">deductive habitat model</a> to remotely-sensed data layers within a species\' range.',
 u'citation': u'U.S. Geological Survey - Gap Analysis Program, 2017, Ring-necked Snake (Diadophis punctatus) Habitat Map, U.S. Geological Survey data release, https://doi:10.5066/F7X065H2\n         .',
 u'contacts': [{u'contactType': u'person',
   u'email': u'amckerrow@usgs.gov',
   u'name': u'Dr. Alexa McKerrow',
   u'organization': {u'displayText': u'U.S. Geological Survey/Core Science Analytics, Synthesis, and Libraries'},
   u'primaryLocation': {u'mailAddress': {u'city': u'Raleigh',
     u'country': u'United States',
     u'line1': u'North Carolina State University, Campus Box 7617',
     u'state': u'NC',
     u'zip': u'27695-7617'},
    u'officePhone': u'571-218-5474',
    u'street